# Introduction to Decision Trees Algorithm

To get familiar with the decision trees ML algorithm, I will be working with income dataset, looking at individual income in the United States. The data is from the 1994 census, and contains information on an individual's marital status, age, type of work, and more. The target column, or what we want to predict, is whether individuals make less than or equal to 50k a year, or more than 50k a year.
The data can be downloaded from [the University fo California, Irvin's website](http://archive.ics.uci.edu/ml/datasets/Adult).

In [1]:
import pandas as pd
import math
import numpy as np 

income = pd.read_csv('income.csv', index_col= False)
print(income.head(5))

   age          workclass  fnlwgt   education  education_num  \
0   39          State-gov   77516   Bachelors             13   
1   50   Self-emp-not-inc   83311   Bachelors             13   
2   38            Private  215646     HS-grad              9   
3   53            Private  234721        11th              7   
4   28            Private  338409   Bachelors             13   

        marital_status          occupation    relationship    race      sex  \
0        Never-married        Adm-clerical   Not-in-family   White     Male   
1   Married-civ-spouse     Exec-managerial         Husband   White     Male   
2             Divorced   Handlers-cleaners   Not-in-family   White     Male   
3   Married-civ-spouse   Handlers-cleaners         Husband   Black     Male   
4   Married-civ-spouse      Prof-specialty            Wife   Black   Female   

   capital_gain  capital_loss  hours_per_week  native_country high_income  
0          2174             0              40   United-States   

It can be seen from the data that there are categorical variables such as workclass that have string values. Multiple individuals can share the same string value. The types of work include State-gov, Self-emp-not-inc, Private, and so on. Each of these strings is a label for a category. Another example of a column of categories is sex, where the options are Male and Female.

### Converting Categorical values

Before starting up with decision trees, the categorical variables need to be converted in our data set to numeric variables. This involves assigning a number to each category label, then converting all of the labels in a column to the corresponding numbers.

One strategy is to convert the columns to a categorical type. Under this approach, pandas will display the labels as strings, but internally store them as numbers so we can do computations with them. The numbers aren't always compatible with other libraries like Scikit-learn, though, so it's easier to just do the conversion to numeric upfront.

Use the pandas.Categorical() class from pandas to perform the conversion to numbers.

In [2]:
# Converting all categorical values into number 
columns = ["workclass","education", "marital_status", "occupation", "relationship", "race", "sex", "native_country", "high_income"]

for column in columns:
    cols = pd.Categorical(income[column])
    income[column] = cols.codes
    
income.head(5)

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,high_income
0,39,7,77516,9,13,4,1,1,4,1,2174,0,40,39,0
1,50,6,83311,9,13,2,4,0,4,1,0,0,13,39,0
2,38,4,215646,11,9,0,6,1,4,1,0,0,40,39,0
3,53,4,234721,1,7,2,6,0,2,1,0,0,40,39,0
4,28,4,338409,9,13,2,10,5,2,0,0,0,40,5,0


### Finding the entropy of high_income 

In [3]:
# Finding the probabilities and then calculating the entropy via formula
prob_0 = income[income["high_income"] == 0].shape[0] / income.shape[0]
prob_1 = income[income["high_income"] == 1].shape[0] / income.shape[0]
income_entropy = - (prob_0 * math.log(prob_0, 2) + prob_1 * math.log(prob_1,2))
print(income_entropy)

0.7963839552022132


### Information Gain

Information gain tells which split will reduce entropy the most. We're computing information gain (IG) for a given target variable (T), as well as a given variable we want to split on (A).

To compute it, we first calculate the entropy for T. Then, for each unique value v in the variable A, we compute the number of rows in which A takes on the value v, and divide it by the total number of rows. Next, we multiply the results by the entropy of the rows where A is v. We add all of these subset entropies together, then subtract from the overall entropy to get information gain. To compute information gain, we'll only have to compute entropies for two subsets.

In [4]:
def calc_entropy(column):
    """
    Calculate entropy given a pandas series, list, or numpy array.
    """
    # Compute the counts of each unique value in the column
    counts = np.bincount(column)
    # Divide by the total column length to get a probability
    probabilities = counts / len(column)
    
    # Initialize the entropy to 0
    entropy = 0
    # Loop through the probabilities, and add each one to the total entropy
    for prob in probabilities:
        if prob > 0:
            entropy += prob * math.log(prob, 2)
    
    return -entropy

income_entropy = calc_entropy(income["high_income"])
print('Income entropy: ', income_entropy)
# Splitting by age column by median
median_age = income["age"].median()

left_split = income[income["age"] <= median_age]
right_split = income[income["age"] > median_age]
# calculating entropy using the probabilities of split nodes
age_information_gain = income_entropy - ((left_split.shape[0] / income.shape[0]) * calc_entropy(left_split["high_income"]) 
                                         + ((right_split.shape[0] / income.shape[0]) * calc_entropy(right_split["high_income"])))

print('Age Information Gain:', age_information_gain)

Income entropy:  0.7963839552022132
Age Information Gain: 0.047028661304691965


### Finding the best split 

We'll find the variable to split on by calculating which split would have the highest information gain.

In [5]:
def calc_information_gain(data, split_name, target_name):
    """
    Calculate information gain given a data set, column to split on, and target
    """
    # Calculate the original entropy
    original_entropy = calc_entropy(data[target_name])
    
    # Find the median of the column we're splitting
    column = data[split_name]
    median = column.median()
    
    # Make two subsets of the data, based on the median
    left_split = data[column <= median]
    right_split = data[column > median]
    
    # Loop through the splits and calculate the subset entropies
    to_subtract = 0
    for subset in [left_split, right_split]:
        prob = (subset.shape[0] / data.shape[0]) 
        to_subtract += prob * calc_entropy(subset[target_name])
    
    # Return information gain
    return original_entropy - to_subtract

# Verify that our answer is the same as on the last screen
print(calc_information_gain(income, "age", "high_income"))

columns = ["age", "workclass", "education_num", "marital_status", "occupation", "relationship", "race", "sex", "hours_per_week", "native_country"]
# Loop through and compute information gain
information_gains = []
for col in columns:
    information_gain = calc_information_gain(income, col, "high_income")
    information_gains.append(information_gain)

# Find the name of the column with the highest gain
highest_gain_index = information_gains.index(max(information_gains))
highest_gain = columns[highest_gain_index]
print(highest_gain)

0.047028661304691965
marital_status


## Building a Decision Tree

We'll use the ID3 Algorithm for constructing decision trees to accomplish this. This algorithm involves recursion and an understanding of time complexity.In general, recursion is the process of splitting a large problem into smaller chunks. Recursive functions will call themselves, then combine the results into a final output.

Building a tree is a perfect use case for recursive algorithms. At each node, we'll call a recursive function that will split the data into two branches. Each branch will lead to a node, and the function will call itself to build the tree out.

Pseudo-code for ID3 algorithm :

def id3(data, target, columns) 

    1 Create a node for the tree
    2 If all values of the target attribute are 1, Return the node, with label = 1
    3 If all values of the target attribute are 0, Return the node, with label = 0
    4 Using information gain, find A, the column that splits the data best
    5 Find the median value in column A
    6 Split column A into values below or equal to the median (0), and values above the median (1)
    7 For each possible value (0 or 1), vi, of A,
    8    Add a new tree branch below Root that corresponds to rows of data where A = vi
    9    Let Examples(vi) be the subset of examples that have the value vi for A
    10    Below this new branch add the subtree id3(data[A==vi], target, columns)
    11 Return Root
   

### Determining the column to split on 


In [6]:
# A list of columns to potentially split income with
columns = ["age", "workclass", "education_num", "marital_status", "occupation", "relationship", "race", "sex", "hours_per_week", "native_country"]

def find_best_column(data, target_name, columns):
    '''
    data is a dataframe
    target_name is the name of the target variable
    columns is a list of potential columns to split on
    
    '''
    information_gains = []
    # Loop through and compute information gains
    for col in columns:
        information_gain = calc_information_gain(data, col, "high_income")
        information_gains.append(information_gain)
        
     #Find the name of the colulm with the highest gain
    highest_gain_index = information_gains.index(max(information_gains))
    highest_gain = columns[highest_gain_index]
    return highest_gain

income_split = find_best_column(income, "high_income", columns)
print(income_split)

marital_status


### Creating a simple recursive algorithm
Let's build up to making the full id3() function by creating a simpler algorithm that we can extend. Here's what that algorithm looks like in pseudocode:

def id3(data, target, columns)
    
    1 Create a node for the tree
    2 If all values of the target attribute are 1, add 1 to counter_1
    3 If all values of the target attribute are 0, add 1 to counter_0
    4 Using information gain, find A, the column that splits the data best
    5 Find the median value in column A
    6 Split A into values below or equal to the median (0), and values above the median (1)
    7 For each possible value (0 or 1), vi, of A,
    8    Add a new tree branch below Root that corresponds to rows of data where A = vi
    9    Let Examples(vi) be the subset of examples that have the value vi for A
    10    Below this new branch, add the subtree id3(data[A==vi], target, columns)
    11 Return Root

In [7]:
label_1s = []
label_0s = []

def id3(data, target, columns):
    # The pandas.unique method will return a list of all the unique values in a series
    unique_targets = pd.unique(data[target])
    
    if len(unique_targets) == 1:
        # See lines 2 and 3 in the algorithm
        # Returning here is critical -- if we don't, the recursive tree will never finish, and run forever
        if 0 in unique_targets:
            label_0s.append(0)
        elif 1 in unique_targets:
            label_1s.append(1)
        return
    
    # Find the best column to split on in our data
    best_column = find_best_column(data, target, columns)
    # Find the median of the column
    column_median = data[best_column].median()
    
    # Create the two splits
    left_split = data[data[best_column] <= column_median]
    right_split = data[data[best_column] > column_median]
    
    # Loop through the splits and call id3 recursively
    for split in [left_split, right_split]:
        # Call id3 recursively to process each branch
        id3(split, target, columns)
        
# Create the data set that we used in the example on the last screen
data = pd.DataFrame([
    [0,20,0],
    [0,60,2],
    [0,40,1],
    [1,25,1],
    [1,35,2],
    [1,55,1]
    ])
# Assign column names to the data
data.columns = ["high_income", "age", "marital_status"]

# Call the function on our data to set the counters properly
id3(data, "high_income", ["age", "marital_status"])


### Storing the tree

Now we can store the entire tree, rather than the leaf labels only. We'll use nested dictionaries to do this. We can represent the root node with a dictionary, and branches with the keys left and right. We'll store the column we're splitting on as the key column, and the median value as the key median. Finally, we can store the label for a leaf as the key label. We'll also number each node as we go along using the number key.
n order to keep track of the tree, we'll need to make some modifications to id3(). The first modification involves changing the definition to pass in the tree dictionary:

def id3(data, target, columns, tree)

    1 Create a node for the tree
    2 Number the node
    3 If all of the values of the target attribute are 1, assign 1 to the label key in tree
    4 If all of the values of the target attribute are 0, assign 0 to the label key in tree
    5 Using information gain, find A, the column that splits the data best
    6 Find the median value in column A
    7 Assign the column and median keys in tree
    8 Split A into values less than or equal to the median (0), and values above the median (1)
    9 For each possible value (0 or 1), vi, of A,
    10    Add a new tree branch below Root that corresponds to rows of data where A = vi
    11    Let Examples(vi) be the subset of examples that have the value vi for A
    12    Create a new key with the name corresponding to the side of the split (0=left, 1=right).  The value of this key should be an empty dictionary.
    13    Below this new branch, add the subtree id3(data[A==vi], target, columns, tree[split_side])
    14 Return Root
    
Under this approach, we're now passing the tree dictionary into our id3 function and setting some keys on it. One complexity is in how we're creating the nested dictionary. For the left split, we're adding a key to the tree dictionary that looks like this:

tree["left"] = {}

For the right side, we're adding:

tree["right"] = {}

Now that we've added this key, we're able to pass our new dictionary into the recursive call to id3(). While this new dictionary will be the dictionary for that specific node, it will be tied back to the parent dictionary (because it's a key of the original dictionary).

This process will continue building up the nested dictionary. We'll be able to access the entire dictionary using the variable tree we define before the function. Think of each recursive call as building a piece of the tree, which we can then access after all of the functions have terminated.

In [8]:
# Create a dictionary to hold the tree  
# It has to be outside of the function so we can access it later
tree = {}

# This list will let us number the nodes  
# It has to be a list so we can access it inside the function
nodes = []

def id3(data, target, columns, tree):
    unique_targets = pd.unique(data[target])
    
    # Assign the number key to the node dictionary
    nodes.append(len(nodes) + 1)
    tree["number"] = nodes[-1]

    if len(unique_targets) == 1:
        # Insert code here that assigns the "label" field to the node dictionary
        if 0 in unique_targets:
            tree["label"] = 0
        elif 1in unique_targets:
            tree["label"] = 1
        return
        
    best_column = find_best_column(data, target, columns)
    column_median = data[best_column].median()
    
    # Insert code here that assigns the "column" and "median" fields to the node dictionary
    tree["column"] = best_column
    tree["median"] = column_median
    
    left_split = data[data[best_column] <= column_median]
    right_split = data[data[best_column] > column_median]
    split_dict = [["left", left_split], ["right", right_split]]
    
    for name, split in split_dict:
        tree[name] = {}
        id3(split, target, columns, tree[name])

# Call the function on our data to set the counters properly
id3(data, "high_income", ["age", "marital_status"], tree)
print(tree)

{'number': 1, 'column': 'age', 'median': 37.5, 'left': {'number': 2, 'column': 'age', 'median': 25.0, 'left': {'number': 3, 'column': 'age', 'median': 22.5, 'left': {'number': 4, 'label': 0}, 'right': {'number': 5, 'label': 1}}, 'right': {'number': 6, 'label': 1}}, 'right': {'number': 7, 'column': 'age', 'median': 55.0, 'left': {'number': 8, 'column': 'age', 'median': 47.5, 'left': {'number': 9, 'label': 0}, 'right': {'number': 10, 'label': 1}}, 'right': {'number': 11, 'label': 0}}}


### Printing label for more attractive tree

The tree dictionary shows all of the relevant information, but it doesn't look very nice. We can fix its appearance by printing it out in a nicer format.

To do this, we'll need to recursively iterate through our tree dictionary. Any dictionary that has a label key is a leaf. Whenever we find one, we'll print out the label. Otherwise, we'll loop through the tree's left and right keys and recursively call the same function.

We also need to keep track of a depth variable. This variable will allow us to use indentation to indicate the order of the nodes. Before we print anything out, we'll prefix it with the number of spaces corresponding to the depth variable.

Here's the pseudocode:

def print_node(tree, depth):
    
    1 Check for the presence of the "label" key in the tree
    2     If found, print the label and return
    3 Print out the tree's "column" and "median" keys
    4 Iterate through the tree's "left" and "right" keys
    5     Recursively call print_node(tree[key], depth+1)

In [9]:
def print_with_depth(string, depth):
    # Add space before a string
    prefix = "    " * depth
    # Print a string, and indent it appropriately
    print("{0}{1}".format(prefix, string))
    
    
def print_node(tree, depth):
    # Check for the presence of "label" in the tree
    if "label" in tree:
        # If found, then this is a leaf, so print it and return
        print_with_depth("Leaf: Label {0}".format(tree["label"]), depth)
        # This is critical -- without it, you'll get infinite recursion
        return
    # Print information about what the node is splitting on
    print_with_depth("{0} > {1}".format(tree["column"], tree["median"]), depth)
    
    # Create a list of tree branches
    branches = [tree["left"], tree["right"]]
        
    for branch in branches:
        print_node(branch, depth+1)

print_node(tree, 0)

age > 37.5
    age > 25.0
        age > 22.5
            Leaf: Label 0
            Leaf: Label 1
        Leaf: Label 1
    age > 55.0
        age > 47.5
            Leaf: Label 0
            Leaf: Label 1
        Leaf: Label 0


### Making predictions automatically

Let's write a function that makes predictions automatically. All we need to do is follow the split points we've already defined with a new row.

Here's the pseudocode:

def predict(tree, row):
    
    1 Check for the presence of "label" in the tree dictionary
    2    If found, return tree["label"]
    3 Extract tree["column"] and tree["median"]
    4 Check whether row[tree["column"]] is less than or equal to tree["median"]
    5    If it's less than or equal, call predict(tree["left"], row) and return the result
    6    If it's greater, call predict(tree["right"], row) and return the result
    
The major difference here is that we're returning values. Because we're only calling the function recursively once in each iteration (we only go "down" a single branch), we can return a single value up the chain of recursion. This will let us get a value back when we call the function.

In [10]:
def predict(tree, row):
    if "label" in tree:
        return tree["label"]
    
    column = tree["column"]
    median = tree["median"]
    
    if row[column] <= median:
        return predict(tree["left"], row)
    else:
        return predict(tree["right"], row)
    
# Print the prediction for the first row in our data
print(predict(tree, data.iloc[0]))

0


In [11]:
## Making multiple predictions
new_data = pd.DataFrame([
    [40,0],
    [20,2],
    [80,1],
    [15,1],
    [27,2],
    [38,1]
    ])
# Assign column names to the data
new_data.columns = ["age", "marital_status"]

def batch_predict(tree, df):
    # Insert your code here
    return df.apply(lambda x: predict(tree,x), axis=1)

predictions = batch_predict(tree, new_data)
print(predictions)

0    0
1    0
2    0
3    0
4    1
5    0
dtype: int64


## Applying Decision trees using scikit-learn 

We can use the scikit-learn package to fit a decision tree. The interface is very similar to other algorithms we've fit in the past.

We use the DecisionTreeClassifier class for classification problems, and DecisionTreeRegressor for regression problems. The sklearn.tree package includes both of these classes.

In this case, we're predicting a binary outcome, so we'll use a classifier.

The first step is to train the classifier on the data. We'll use the fit method on a classifier to do this.


In [12]:
from sklearn.tree import DecisionTreeClassifier

# A list of columns to train with
# We've already converted all columns to numeric
columns = ["age", "workclass", "education_num", "marital_status", "occupation", "relationship", "race", "sex", "hours_per_week", "native_country"]

# Instantiate the classifier
# Set random_state to 1 to make sure the results are consistent
clf = DecisionTreeClassifier(random_state=1)

# We've already loaded the variable "income," which contains all of the income data
clf.fit(income[columns], income["high_income"])

DecisionTreeClassifier(random_state=1)

### Splitting the data in train and test sets

Now that we've fit a model, we can make predictions. We'll want to split our data into training and testing sets first. If we don't, we'll be making predictions on the same data that we train our algorithm with. This leads to overfitting, and will make our error appear lower than it is.

In this case, we'll make 80% of our rows training data, and the rest testing data.

In [13]:
# Set a random seed so the shuffle is the same every time
np.random.seed(1)

# Shuffle the rows  
# This permutes the index randomly using numpy.random.permutation
# Then, it reindexes the dataframe with the result
# The net effect is to put the rows into random order
income = income.reindex(np.random.permutation(income.index))

train_max_row = math.floor(income.shape[0] * .8)

train  = income[0: train_max_row]
test = income[train_max_row:]
print(train.shape)
print(test.shape)

(26048, 15)
(6513, 15)


### Evaluating errors with AUC

AUC ranges from 0 to 1, so it's ideal for binary classification. The higher the AUC, the more accurate our predictions.

We can compute AUC with the roc_auc_score function from sklearn.metrics. This function takes in two parameters:

y_true: true labels

y_score: predicted labels

It then calculates and returns the AUC value.

In [14]:
from sklearn.metrics import roc_auc_score

clf = DecisionTreeClassifier(random_state=1)
clf.fit(train[columns], train["high_income"])

predictions = clf.predict(test[columns])
error = roc_auc_score(test["high_income"], predictions)
print(error)


0.6934656324746192


The AUC for the predictions on the testing set is about .694. Let's compare this against the AUC for predictions on the training set to see if the model is overfitting.

It's normal for the model to predict the training set better than the testing set. After all, it has full knowledge of that data and the outcomes. However, if the AUC between training set predictions and actual values is significantly higher than the AUC between test set predictions and actual values, it's a sign that the model may be overfitting.

In [15]:
predictions = clf.predict(train[columns])
print(roc_auc_score(train["high_income"], predictions))

0.9471244501437455


Our AUC on the training set was .947, and the AUC on the test set was .694. There's no hard and fast rule on when overfitting is occurring, but our model is predicting the training set much better than the test set. Splitting the data into training and testing sets doesn't prevent overfitting -- it just helps us detect and fix it.

Based on our AUC measurements, it appears that we are in fact overfitting. 

Trees overfit when they have too much depth and make overly complex rules that match the training data, but aren't able to generalize well to new data. This may seem to be a strange principle at first, but the deeper a tree is, the worse it typically performs on new data.

### Reducing overfitting with a shallower tree
There are three main ways to combat overfitting:

1. "Prune" the tree after we build it to remove unnecessary leaves.
2. Use ensembling to blend the predictions of many trees.
3. Restrict the depth of the tree while we're building it.

#### Limiting tree depth during the building process
We can restrict tree depth by adding a few parameters when we initialize the DecisionTreeClassifier class:

- max_depth - Globally restricts how deep the tree can go
- min_samples_split - The minimum number of rows a node should have before it can be split; if this is set to 2, for example, then nodes with 2 rows won't be split, and will become leaves instead
- min_samples_leaf - The minimum number of rows a leaf must have
- min_weight_fraction_leaf - The fraction of input rows a leaf must have
- max_leaf_nodes - The maximum number of total leaves; this will cap the count of leaf nodes as the tree is being built

Some of these parameters aren't compatible, however. For example, we can't use max_depth and max_leaf_nodes together.

In [16]:
# Decision trees model by reducing the tree depth or sample split
clf = DecisionTreeClassifier(random_state=1, min_samples_split = 13)

clf.fit(train[columns], train["high_income"])

train_auc = roc_auc_score(train["high_income"], clf.predict(train[columns]))
test_auc = roc_auc_score(test["high_income"], clf.predict(test[columns]))
print(train_auc)
print(test_auc)

0.8421431849275413
0.6995617145150872


By setting min_samples_split to 13, we managed to boost the test AUC from .694 to .710. The training set AUC decreased from .947 to .836, showing that the model we built was less overfit to the training set than before.

In [17]:
# The first decision trees model we trained and tested; tweaking paarameters to adjust auc
clf = DecisionTreeClassifier(random_state=1, max_depth = 7, min_samples_split = 13)
clf.fit(train[columns], train["high_income"])
predictions = clf.predict(test[columns])
test_auc = roc_auc_score(test["high_income"], predictions)

train_predictions = clf.predict(train[columns])
train_auc = roc_auc_score(train["high_income"], train_predictions)

print(test_auc)
print(train_auc)

0.7436344996725136
0.748037708309209


Let's tweak those parameters more aggresively to adjust AUC by increasing samples split to 100

In [18]:
# The first decision trees model we trained and tested; tweaking paarameters to adjust auc
clf = DecisionTreeClassifier(random_state=1, max_depth = 2, min_samples_split = 100)
clf.fit(train[columns], train["high_income"])
predictions = clf.predict(test[columns])
test_auc = roc_auc_score(test["high_income"], predictions)

train_predictions = clf.predict(train[columns])
train_auc = roc_auc_score(train["high_income"], train_predictions)

print(test_auc)
print(train_auc)

0.6553138481876499
0.6624508042161483


Our accuracy went down on the last screen, relative to the screen before it:

|settings	|train AUC	|test AUC|
|-----------|-----------|--------|
|default (min_samples_split: 2, max_depth: None)|	0.947	|0.694|
|min_samples_split: 13	|0.836	|0.710|
|min_samples_split: 13, max_depth: 7	|0.750	|0.749|
|min_samples_split: 100, max_depth: 2	|0.666	|0.659|

This is because we're now underfitting. Underfitting is what occurs when our model is too simple to explain the relationships between the variables.
#### Bias-Variance tradeoff

By artificially restricting the depth of our tree, we prevent it from creating a model that's complex enough to correctly categorize some of the rows. If we don't perform the artificial restrictions, however, the tree becomes too complex, fits quirks in the data that only exist in the training set, and doesn't generalize to new data.

This is known as the bias-variance tradeoff. Imagine that we take a random sample of training data and create many models. If the models' predictions for the same row are far apart from each other, we have high variance. Imagine this time that we take a random sample of the training data and create many models. If the models' predictions for the same row are close together but far from the actual value, then we have high bias.

High bias can cause underfitting -- if a model is consistently failing to predict the correct value, it may be that it's too simple to model the data faithfully.

High variance can cause overfitting. If a model varies its predictions significantly based on small changes in the input data, then it's likely fitting itself to quirks in the training data, rather than making a generalizable model.

We call this the bias-variance tradeoff because decreasing one characteristic will usually increase the other. This is a limitation of all machine learning algorithms.

Decision trees typically suffer from high variance. The entire structure of a decision tree can change if we make a minor alteration to its training data. By restricting the depth of the tree, we increase the bias and decrease the variance. If we restrict the depth too much, we increase bias to the point where it will underfit.

You'll generally need to use your intuition and manually tweak parameters to get the "right" fit.

#### Pruning 
One way to prevent overfitting is to block the tree from growing beyond a certain depth (we tried this before). Another technique is called pruning. Pruning involves building a full tree, and then removing the leaves that don't add to prediction accuracy. Pruning prevents a model from becoming overly complex. It can result in a simpler model that has higher accuracy on the testing set.


### Advantages and Disadvantages of Decision Trees algorithm 

The main advantages of using decision trees is that they're:

- Easy to interpret
- Relatively fast to fit and make predictions
- Able to handle multiple types of data
- Able to pick up nonlinearities in data, and usually fairly accurate

The main disadvantage of using decision trees is their tendency to overfit.

Decision trees are a good choice for tasks where it's important to be able to interpret and convey why the algorithm is doing what it's doing.

The most powerful way to reduce decision tree overfitting is to create ensembles of trees. The random forest algorithm is a popular choice for doing this. In cases where prediction accuracy is the most important consideration, random forests usually perform better.

## Random Forests Algorithm 
The most powerful tool for reducing decision tree overfitting is called the random forest algorithm. A random forest is a kind of ensemble model. Ensembles combine the predictions of multiple models to create a more accurate final prediction.

Let's create two decision trees with slightly different parameters:

- One with min_samples_leaf set to 2
- One with max_depth set to 5

Then, we'll check their accuracies separately. 

In [19]:
columns = ["age", "workclass", "education_num", "marital_status", "occupation", "relationship", "race", "sex", "hours_per_week", "native_country"]

clf = DecisionTreeClassifier(random_state=1, min_samples_leaf=2)
clf.fit(train[columns], train["high_income"])

clf2 = DecisionTreeClassifier(random_state=1, max_depth=5)
clf2.fit(train[columns], train["high_income"])

clf_predictions = clf2.predict(test[columns])
clf2_predictions = clf2.predict(test[columns])

auc_clf = roc_auc_score(test["high_income"], clf_predictions)
auc_clf2 = roc_auc_score(test["high_income"], clf2_predictions)

print(auc_clf)
print(auc_clf2)


0.6759853906508785
0.6759853906508785


In [20]:
predictions = clf.predict_proba(test[columns])[:,1]
predictions2 = clf2.predict_proba(test[columns])[:,1]

mean = np.round((predictions + predictions2) / 2)
print(roc_auc_score(test["high_income"], mean))

0.7150846804038882


### Why ensembling works

The models are approaching the same problem in slightly different ways, and building different trees because we used different parameters for each one. Each tree makes different predictions in different areas. Even though both trees have about the same accuracy, when we combine them, the result is stronger because it leverages the strengths of both approaches.

Ensembling a decision tree and a logistic regression model, for example, will result in stronger predictions than ensembling two decision trees with similar parameters. That's because those two models use very different approaches to arrive at their answers.

On the other hand, if the models we ensemble are very similar in how they make predictions, ensembling will result in a negligible boost.


## Introducing variation with bagging 

A random forest is an ensemble of decision trees. If we don't make any modifications to the trees, each tree will be exactly the same, so we'll get no boost when we ensemble them. In order to make ensembling effective, we have to introduce variation into each individual decision tree model.

If we introduce variation, each tree will be be constructed slightly differently, and will therefore make different predictions. This variation is what puts the "random" in "random forest."

There are two main ways to introduce variation in a random forest -- bagging and random feature subsets. 

In a random forest, we don't train each tree on the entire data set. We train it on a random sample of the data, or a "bag," instead. We perform this sampling with replacement, which means that after we select a row from the data we're sampling, we put the row back in the data so it can be picked again. Some rows from the original data may appear in the "bag" multiple times.

In [21]:
# We'll build 10 trees
tree_count = 10

# Each "bag" will have 60% of the number of original rows
bag_proportion = .6

predictions = []
for i in range(tree_count):
    # We select 60% of the rows from train, sampling with replacement
    # We set a random state to ensure we'll be able to replicate our results
    # We set it to i instead of a fixed value so we don't get the same sample in every loop
    # That would make all of our trees the same
    bag = train.sample(frac=bag_proportion, replace=True, random_state=i)
    
    # Fit a decision tree model to the "bag"
    clf = DecisionTreeClassifier(random_state=1, min_samples_leaf=2)
    clf.fit(bag[columns], bag["high_income"])
    
    # Using the model, make predictions on the test data
    predictions.append(clf.predict_proba(test[columns])[:,1])

combined = np.sum(predictions, axis=0) / 10
rounded = np.round(combined)

print(roc_auc_score(test["high_income"], rounded))
print(predictions)

0.7329963297474371
[array([0.25      , 0.33333333, 0.        , ..., 0.        , 0.66666667,
       0.33333333]), array([0.25, 0.  , 0.  , ..., 0.  , 0.  , 0.  ]), array([0.25, 0.  , 0.  , ..., 0.  , 1.  , 1.  ]), array([0. , 0. , 0. , ..., 0. , 0. , 0.5]), array([0.28571429, 0.        , 0.66666667, ..., 0.        , 0.        ,
       0.25      ]), array([0.        , 0.        , 0.        , ..., 0.66666667, 1.        ,
       0.5       ]), array([0.25      , 0.        , 0.        , ..., 1.        , 0.66666667,
       0.66666667]), array([0. , 0. , 0. , ..., 0. , 0. , 0.5]), array([0.25, 0.  , 0.  , ..., 0.  , 1.  , 1.  ]), array([0.33333333, 0.        , 0.66666667, ..., 1.        , 0.        ,
       0.5       ])]


### Selecting random features

Using the bagging example, we gained some accuracy over a single decision tree. To be exact, we achieved an AUC score of around .733 with bagging, which is an improvement over the AUC score of .688 we got without bagging.

Let's go back to the decision tree algorithm we explored earlier to explain random feature subsets:

- First we pick the maximum number of features we want to evaluate each time we split the tree.
    - This has to be less than the total number of columns in the data.
- Every time we split, we pick a random sample of features from the data.
- Then we compute the information gain for each feature in our random sample, and pick the one with the highest information gain to split on.

To understand how splits work, let's look at information gain or entropy. Entropy is the measure of "disorder" in the data set. If a dataset has all the same labels, they'll have low entropy. If all the labels are different, they'll have high entropy. Splits that give us more information about the data, will ideally minimize entropy. In other words, the tree will ideally split the labels into distinct groups with as little mixture possible. This'll allow the splits to give our tree more predictive power.

We're repeating the same process to select the optimal split that minimizes entropy for a node. However, we'll only evaluate a constrained set of features that we select randomly. This introduces variation into the trees, and makes for more powerful ensembles.


In [22]:
# Create the data set that we used two missions ago
data = pd.DataFrame([
    [0,4,20,0],
    [0,4,60,2],
    [0,5,40,1],
    [1,4,25,1],
    [1,5,35,2],
    [1,5,55,1]
    ])
data.columns = ["high_income", "employment", "age", "marital_status"]

# Set a random seed to make the results reproducible
np.random.seed(1)

# The dictionary to store our tree
tree = {}
nodes = []

# The function to find the column to split on
def find_best_column(data, target_name, columns):
    information_gains = []
    
    # insert your code here
    
    for col in columns:
        information_gain = calc_information_gain(data, col, "high_income")
        information_gains.append(information_gain)

    # Find the name of the column with the highest gain
    highest_gain_index = information_gains.index(max(information_gains))
    highest_gain = columns[highest_gain_index]
    return highest_gain
# The function to construct an ID3 decision tree
def id3(data, target, columns, tree):
    unique_targets = pd.unique(data[target])
    nodes.append(len(nodes) + 1)
    tree["number"] = nodes[-1]

    if len(unique_targets) == 1:
        if 0 in unique_targets:
            tree["label"] = 0
        elif 1 in unique_targets:
            tree["label"] = 1
        return
    
    best_column = find_best_column(data, target, columns)
    column_median = data[best_column].median()
    
    tree["column"] = best_column
    tree["median"] = column_median
    
    left_split = data[data[best_column] <= column_median]
    right_split = data[data[best_column] > column_median]
    split_dict = [["left", left_split], ["right", right_split]]
    
    for name, split in split_dict:
        tree[name] = {}
        id3(split, target, columns, tree[name])
# Run the ID3 algorithm on our data set and print the resulting tree
id3(data, "high_income", ["employment", "age", "marital_status"], tree)
print(tree)
def find_best_column(data, target_name, columns):
    information_gains = []
    
    # Select two columns randomly
    cols = np.random.choice(columns, 2)
    
    for col in cols:
        information_gain = calc_information_gain(data, col, "high_income")
        information_gains.append(information_gain)

    highest_gain_index = information_gains.index(max(information_gains))
    
    # Get the highest gain by indexing "cols"
    highest_gain = cols[highest_gain_index]
    
    return highest_gain

id3(data, "high_income", ["employment", "age", "marital_status"], tree)
print(tree)

{'number': 1, 'column': 'employment', 'median': 4.5, 'left': {'number': 2, 'column': 'age', 'median': 25.0, 'left': {'number': 3, 'column': 'age', 'median': 22.5, 'left': {'number': 4, 'label': 0}, 'right': {'number': 5, 'label': 1}}, 'right': {'number': 6, 'label': 0}}, 'right': {'number': 7, 'column': 'age', 'median': 40.0, 'left': {'number': 8, 'column': 'age', 'median': 37.5, 'left': {'number': 9, 'label': 1}, 'right': {'number': 10, 'label': 0}}, 'right': {'number': 11, 'label': 1}}}
{'number': 12, 'column': 'age', 'median': 37.5, 'left': {'number': 13, 'column': 'employment', 'median': 4.0, 'left': {'number': 14, 'column': 'age', 'median': 22.5, 'left': {'number': 15, 'label': 0}, 'right': {'number': 16, 'label': 1}}, 'right': {'number': 17, 'label': 1}}, 'right': {'number': 18, 'column': 'age', 'median': 55.0, 'left': {'number': 19, 'column': 'age', 'median': 47.5, 'left': {'number': 20, 'label': 0}, 'right': {'number': 21, 'label': 1}}, 'right': {'number': 22, 'label': 0}}}


### Random subsets in scikit-learn 
We can also repeat our random subset selection process in scikit-learn. We just set the splitter parameter on DecisionTreeClassifier to "random", and the max_features parameter to "auto". If we have N columns, this will pick a subset of features of size √N, compute the Gini coefficient for each (this is similar to information gain), and split the node on the best column in the subset.


In [24]:
# We'll build 10 trees
tree_count = 10

# Each "bag" will have 60% of the number of original rows
bag_proportion = .6

predictions = []
for i in range(tree_count):
    # We select 60% of the rows from train, sampling with replacement
    # We set a random state to ensure we'll be able to replicate our results
    # We set it to i instead of a fixed value so we don't get the same sample every time
    bag = train.sample(frac=bag_proportion, replace=True, random_state=i)
    
    # Fit a decision tree model to the "bag"
    clf = DecisionTreeClassifier(random_state=1, min_samples_leaf=2, splitter = "random", max_features = "auto")
    clf.fit(bag[columns], bag["high_income"])
    
    # Using the model, make predictions on the test data
    predictions.append(clf.predict_proba(test[columns])[:,1])

combined = np.sum(predictions, axis=0) / 10
rounded = np.round(combined)

print(roc_auc_score(test["high_income"], rounded))

0.7345958637997538


So far, we've demonstrated the two building blocks of random forests, bagging and random feature subsets. Luckily, we don't have to write code from scratch each time. Scikit-learn has a RandomForestClassifier class and a RandomForestRegressor class that enable us to train and test random forest models quickly.

When we instantiate a RandomForestClassifier, we pass in an n_estimators parameter that indicates how many trees to build. While adding more trees usually improves accuracy, it also increases the overall time the model takes to train.

RandomForestClassifier has a similar interface to DecisionTreeClassifier, and we can use the fit() and predict() methods to train and make predictions.

In [25]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(n_estimators=5, random_state=1, min_samples_leaf=2)
clf.fit(train[columns], train["high_income"])

predictions = clf.predict(test[columns])
print(roc_auc_score(test["high_income"], predictions))

0.7347461391939776


## Tweaking parameters to increase accuracy

Similar to decision trees, we can tweak some of the parameters for random forests, including:

- min_samples_leaf
- min_samples_split
- max_depth
- max_leaf_nodes

These parameters apply to the individual trees in the model, and change how they are constructed. There are also parameters specific to the random forest that alter its overall construction:

- n_estimators
- bootstrap 

"Bootstrap aggregation" is another name for bagging; this parameter indicates whether to turn it on (Defaults to True)

Tweaking parameters can increase the accuracy of the forest. The easiest tweak is to increase the number of estimators we use. This approach yields diminishing returns -- going from 10 trees to 100 will make a bigger difference than going from 100 to 500, which will make a bigger difference than going from 500 to 1000. The accuracy increase function is logarithmic, so increasing the number of trees beyond a certain number (usually 200) won't help much at all.

In [26]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(n_estimators=150, random_state=1, min_samples_leaf=2)
clf.fit(train[columns], train["high_income"])

predictions = clf.predict(test[columns])
print(roc_auc_score(test["high_income"], predictions))

0.7379403213124711


## Reducing overfitting 

One of the major advantages of random forests over single decision trees is that they tend to overfit less. Although each individual decision tree in a random forest varies widely, the average of their predictions is less sensitive to the input data than a single tree is. This is because while one tree can construct an incorrect and overfit model, the average of 100 or more trees will be more likely to hone in on the signal and ignore the noise. The signal will be the same across all of the trees, whereas each tree will hone in on the noise differently. This means that the average will discard the noise and keep the signal.



In [27]:
clf = DecisionTreeClassifier(random_state=1, min_samples_leaf=5)

clf.fit(train[columns], train["high_income"])

predictions = clf.predict(train[columns])
print("AUC without estimators for train dataset: ", roc_auc_score(train["high_income"], predictions))

predictions = clf.predict(test[columns])
print("AUC without estimators for test dataset: ", roc_auc_score(test["high_income"], predictions))

clf = RandomForestClassifier(n_estimators=150, random_state=1, min_samples_leaf=5)

clf.fit(train[columns], train["high_income"])

predictions = clf.predict(train[columns])
print("AUC with estimators for train dataset: ", roc_auc_score(train["high_income"], predictions))

predictions = clf.predict(test[columns])
print("AUC with estimators for test dataset: ", roc_auc_score(test["high_income"], predictions))

AUC without estimators for train dataset:  0.8192570489534683
AUC without estimators for test dataset:  0.7139325899284541
AUC with estimators for train dataset:  0.7917047295143252
AUC with estimators for test dataset:  0.7498874343962398


We've fit a single decision tree to the training set, and made predictions for both the training and testing sets. The AUC for the training set predictions is .819, while the AUC for the testing set is .714. The fact that the test AUC is much lower than the train AUC indicates that the model is overfitting.

## When to use Random Forests

overfitting decreased with a random forest, and accuracy went up overall.

While the random forest algorithm is incredibly powerful, it isn't applicable to all tasks. The main strengths of a random forest are:

- Very accurate predictions - Random forests achieve near state-of-the-art performance on many machine learning tasks. Along with neural networks and gradient-boosted trees, they're typically one of the top-performing algorithms.
- Resistance to overfitting - Due to their construction, random forests are fairly resistant to overfitting. We still need to set and tweak parameters like max_depth though.

The main weaknesses of using a random forest are:

- They're difficult to interpret - Because we're averaging the results of many trees, it can be hard to figure out why a random forest is making predictions the way it is.
- They take longer to create - Making two trees takes twice as long as making one, making three takes three times as long, and so on. Fortunately, we can exploit multicore processors to parallelize tree construction. Scikit allows us to do this through the n_jobs parameter on RandomForestClassifier. We'll discuss parallelization in greater detail later on.

Given these trade-offs, it makes sense to use random forests in situations where accuracy is of the utmost importance; being able to interpret or explain the decisions the model is making isn't key. In cases where time is of the essence or interpretability is important, a single decision tree may be a better choice.